In [1]:
import os
import numpy as np
import pandas as pd
from utils import model_utils

from signlens.params import *
from signlens.preprocessing.data import load_data_subset_csv, load_glossary, load_video_list_json, load_landmarks_json
from signlens.preprocessing.preprocess import group_pad_sequences, encode_labels, decode_labels, pad_and_preprocess_sequence, reshape_processed_data_to_tf
from utils.model_utils import load_model
from utils.video_utils import process_video_to_landmarks_json

%load_ext autoreload

%autoreload 2


2024-03-24 00:58:26.596752: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-24 00:58:27.480501: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
test_data = load_data_subset_csv(balanced=True, csv_path=TRAIN_TEST_CSV_PATH)


Loading data subset from train_test.csv
    ℹ️ Filtered sequences with missing frames. Size reduced from 17233 to 17233 (100.00%)
    ℹ️ Filtered on n_frames = 100. Size reduced from 17233 to 17233 (100.00%)
    ℹ️ Filtered on n_classes = 10. Size reduced from 17233 to 690 (4.00%)
    ℹ️ Balanced data, with average of 34.5 elements per class. Size reduced from 690 to 345 (50.00%)
✅ Loaded 345 rows (2.00% of the original 17233 rows) from the dataset.


In [3]:
test_data

,path,participant_id,sequence_id,sign,n_frames,n_frames2,file_path
0,train_landmark_files_noface/16069/2950486636.p...,16069,2950486636,go,52,52,/home/bfrisque/code/benoitfrisque/signlens/raw...
1,train_landmark_files_noface/55372/2509506508.p...,55372,2509506508,yes,92,92,/home/bfrisque/code/benoitfrisque/signlens/raw...
2,train_landmark_files_noface/29302/3046062340.p...,29302,3046062340,who,43,43,/home/bfrisque/code/benoitfrisque/signlens/raw...
3,train_landmark_files_noface/55372/703384408.pa...,55372,703384408,fine,16,16,/home/bfrisque/code/benoitfrisque/signlens/raw...
4,train_landmark_files_noface/22343/1991418745.p...,22343,1991418745,book,8,8,/home/bfrisque/code/benoitfrisque/signlens/raw...
...,...,...,...,...,...,...,...
340,train_landmark_files_noface/22343/2257620665.p...,22343,2257620665,go,16,16,/home/bfrisque/code/benoitfrisque/signlens/raw...
341,train_landmark_files_noface/32319/2015228315.p...,32319,2015228315,book,41,41,/home/bfrisque/code/benoitfrisque/signlens/raw...
342,train_landmark_files_noface/18796/972839332.pa...,18796,972839332,book,18,18,/home/bfrisque/code/benoitfrisque/signlens/raw...
343,train_landmark_files_noface/34503/4223537013.p...,34503,4223537013,no,14,14,/home/bfrisque/code/benoitfrisque/signlens/raw...


# Model predict

In [4]:
model_name = "model 20240322-173411"
model, _ = load_model(model_name)

processed_data = group_pad_sequences(test_data.file_path.iloc[0:2])

prediction = model.predict([processed_data])

word, proba = decode_labels(prediction)

word, proba



Load latest model from local registry...

Load latest model from disk...
✅ Model loaded from local disk /home/bfrisque/code/benoitfrisque/signlens/training_outputs/model 20240322-173411


2024-03-24 00:58:30.261512: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-24 00:58:30.262160: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/bfrisque/.pyenv/versions/3.10.6/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/home/bfrisque/.pyenv/versions/3.10.6/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fo

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step


(['who', 'book'], array([0.15768468, 0.15268382], dtype=float32))

# Video conversion

In [5]:
videos = load_video_list_json()
videos

,sign,bbox,fps,frame_end,frame_start,instance_id,signer_id,source,split,url,variation_id,video_id,video_path
0,airplane,"[605, 21, 1721, 1076]",25,-1,1,10,4,signschool,train,https://signstock.blob.core.windows.net/signsc...,0,01726,/home/bfrisque/code/benoitfrisque/signlens/raw...
1,table,"[374, 52, 810, 720]",25,-1,1,13,38,startasl,train,https://s3-us-west-1.amazonaws.com/files.start...,0,56556,/home/bfrisque/code/benoitfrisque/signlens/raw...
2,see,"[85, 15, 230, 192]",25,-1,1,8,11,signingsavvy,train,https://www.signingsavvy.com/signs/mp4/8/8396.mp4,0,50125,/home/bfrisque/code/benoitfrisque/signlens/raw...
3,who,"[165, 4, 472, 370]",25,-1,1,14,88,aslsignbank,train,https://aslsignbank.haskins.yale.edu/dictionar...,0,66778,/home/bfrisque/code/benoitfrisque/signlens/raw...
4,dog,"[417, 61, 834, 720]",25,-1,1,7,38,startasl,train,https://s3-us-west-1.amazonaws.com/files.start...,1,17086,/home/bfrisque/code/benoitfrisque/signlens/raw...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1471,arm,"[205, 37, 489, 370]",25,-1,1,3,89,aslsignbank,val,https://aslsignbank.haskins.yale.edu/dictionar...,0,65094,/home/bfrisque/code/benoitfrisque/signlens/raw...
1472,say,"[104, 0, 528, 480]",25,-1,1,4,13,asldeafined,val,https://media.asldeafined.com/vocabulary/14687...,0,49430,/home/bfrisque/code/benoitfrisque/signlens/raw...
1473,ear,"[296, 36, 879, 720]",25,-1,1,1,118,aslbrick,train,http://aslbricks.org/New/ASL-Videos/ear.mp4,0,69306,/home/bfrisque/code/benoitfrisque/signlens/raw...
1474,closet,"[64, 14, 260, 240]",25,-1,1,6,26,spreadthesign,train,https://media.spreadthesign.com/video/mp4/13/3...,0,11284,/home/bfrisque/code/benoitfrisque/signlens/raw...


In [6]:
video_path = videos[videos.sign == 'book'].video_path.iloc[0]
video_path


'/home/bfrisque/code/benoitfrisque/signlens/raw_data/WLASL/videos/07070.mp4'

In [7]:
json_data = process_video_to_landmarks_json(video_path, output=True, frame_interval=1)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


✅ Landmarks saved to '/home/bfrisque/code/benoitfrisque/signlens/processed_data/landmarks_videos/landmarks_07070.json'


In [8]:
landmarks_json_path = '/home/bfrisque/code/benoitfrisque/signlens/processed_data/landmarks_videos/landmarks_07070.json'

landmarks = load_landmarks_json(landmarks_json_path)
data_processed = pad_and_preprocess_sequence(landmarks)
data_tf = reshape_processed_data_to_tf(data_processed)
data_tf.shape

TensorShape([1, 100, 225])

In [9]:
model_name = "model 20240322-173411"
model, _ = load_model(model_name)

prediction = model.predict(data_tf)

word, proba = decode_labels(prediction)

word, proba



Load latest model from local registry...

Load latest model from disk...
✅ Model loaded from local disk /home/bfrisque/code/benoitfrisque/signlens/training_outputs/model 20240322-173411
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 690ms/step


(['fine'], array([0.15782131], dtype=float32))